<a href="https://colab.research.google.com/github/abdul9870/abdul9870/blob/main/Tinyllama_qlora_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TinyLlama QLoRA Fine-tuning for Text-to-SQL

This notebook demonstrates how to fine-tune TinyLlama using QLoRA (Quantized Low-Rank Adaptation) on the Spider dataset for text-to-SQL generation. QLoRA is designed to be memory-efficient and work on systems with limited GPU resources.

## Overview

This implementation uses the following techniques:

- **QLoRA**: A memory-efficient fine-tuning approach that quantizes the base model to 4-bit precision and uses Low-Rank Adaptation (LoRA) for parameter-efficient fine-tuning.
- **TinyLlama**: A compact 1.1B parameter language model that offers a good balance between performance and resource requirements.
- **Text-to-SQL**: The model is fine-tuned to convert natural language questions into SQL queries.

## References

### Papers
- [QLoRA: Efficient Finetuning of Quantized LLMs](https://arxiv.org/abs/2305.14314) - Dettmers et al., 2023
- [LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685) - Hu et al., 2021
- [TinyLlama: An Open-Source Small Language Model](https://arxiv.org/abs/2401.02385) - Zhang et al., 2024
- [Spider: A Large-Scale Human-Labeled Dataset for Text-to-SQL Tasks](https://arxiv.org/abs/1809.08887) - Yu et al., 2018

### Blogs and Resources
- [Parameter-Efficient Fine-Tuning of LLMs](https://huggingface.co/blog/peft) - Hugging Face Blog
- [QLoRA: Quantization for LLM Adaptation](https://huggingface.co/blog/4bit-transformers-bitsandbytes) - Hugging Face Blog
- [TinyLlama Project](https://github.com/jzhang38/TinyLlama) - GitHub Repository
- [Text-to-SQL with Transformers](https://huggingface.co/blog/text2sql) - Hugging Face Blog

## Install Required Dependencies

In [ ]:
# Install core dependencies for QLoRA fine-tuning
!pip install -q datasets transformers peft bitsandbytes accelerate tqdm pandas numpy
# Install Weights & Biases for experiment tracking and visualization
!pip install -q wandb
# Install Gradio for creating interactive demo interfaces
!pip install -q gradio
# Install Hugging Face Hub for model sharing
!pip install -q huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

## Import Required Libraries

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import time  # For tracking training time
import logging  # For enhanced logging
from datetime import datetime
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    get_scheduler  # For custom learning rate scheduling
)
from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    PeftModel
)
import wandb
import gradio as gr
from huggingface_hub import login
from tqdm.auto import tqdm

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),  # Output to console
    ]
)
logger = logging.getLogger(__name__)

## Configure Model and Training Parameters

The parameters below have been optimized for a 20-30 minute training session with improved logging and performance.

In [ ]:
import os
import logging
from datetime import datetime

# Model configuration
MODEL_NAME         = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Base model for fine-tuning
OUTPUT_DIR         = "finetuned-tinyllama-spider-qlora"  # Directory to save model checkpoints
HUGGINGFACE_REPO   = "yourusernamehere/tinyllama-spider-sql"  # Change this to your username

# Training parameters - extended training time
MAX_LENGTH                      = 512   # Maximum sequence length for tokenization
BATCH_SIZE                      = 8     # Examples per batch
GRADIENT_ACCUMULATION_STEPS     = 2     # Accumulate gradients over 2 steps
LEARNING_RATE                   = 3e-4  # Base learning rate
NUM_EPOCHS                      = 15    # Increased from 5 → 15 to extend training duration
WARMUP_RATIO                    = 0.1   # 10% of total steps for warmup
WEIGHT_DECAY                    = 0.05  # L2 regularization
LOGGING_STEPS                   = 5     # Log every 5 steps
EVAL_STEPS                      = 20    # Evaluate every 20 steps
SAVE_STEPS                      = 50    # Save checkpoint every 50 steps

# LoRA configuration - unchanged
LORA_R         = 16     # Rank of the update matrices
LORA_ALPHA     = 32     # Scaling factor for the updates
LORA_DROPOUT   = 0.05   # Dropout probability on LoRA layers
TARGET_MODULES = ["q_proj", "v_proj", "k_proj", "o_proj"]  # Target all attention modules

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/logs", exist_ok=True)  # Create logs directory

# Configure file logging
file_handler = logging.FileHandler(f"{OUTPUT_DIR}/logs/training_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
logger.addHandler(file_handler)

## Load and Prepare Spider Dataset

The Spider dataset is a large-scale, complex, and cross-domain semantic parsing and text-to-SQL dataset. It contains 10,181 questions and 5,693 unique complex SQL queries on 200 databases with multiple tables.

In [ ]:
import os
import logging
from datetime import datetime
from datasets import load_dataset, Dataset, DatasetDict

# Configure logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setLevel(logging.INFO)
    handler.setFormatter(
        logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
    )
    logger.addHandler(handler)


def load_spider_dataset():
    """
    Load the Spider dataset using Hugging Face Datasets.
    Falls back to a synthetic dataset if remote loading fails.

    Returns:
        DatasetDict: Contains 'train' and 'validation' splits.
    """
    try:
        logger.info("Loading Spider dataset from Hugging Face...")
        dataset = load_dataset("spider")
        logger.info("Successfully loaded 'spider' dataset.")
        return dataset

    except Exception as e:
        logger.warning(f"Could not load Spider dataset: {e}")
        logger.info("Creating synthetic Spider dataset for demonstration...")

        # Synthetic data examples
        synthetic_data = [
            {"question": "How many students are there?", "query": "SELECT COUNT(*) FROM students;"},
            {"question": "What are the names of all students?", "query": "SELECT name FROM students;"},
            {"question": "Find the average age of students in each department.",
             "query": "SELECT department, AVG(age) FROM students GROUP BY department;"},
            {"question": "List all courses with more than 50 students.",
             "query": "SELECT course_name FROM courses WHERE num_students > 50;"},
            {"question": "Find the department with the highest average GPA.",
             "query": "SELECT department, AVG(gpa) as avg_gpa FROM students GROUP BY department ORDER BY avg_gpa DESC LIMIT 1;"},
        ]

        # Expand to 100 examples for training
        for i in range(len(synthetic_data), 100):
            synthetic_data.append({
                "question": f"Example question {i} about database schema?",
                "query": f"SELECT col{i%5} FROM table{i%3} WHERE cond{i%4} = val{i%6};"
            })

        # Split into train/validation
        split_idx = int(0.8 * len(synthetic_data))
        train_list = synthetic_data[:split_idx]
        val_list = synthetic_data[split_idx:]

        synthetic_dataset = DatasetDict({
            "train": Dataset.from_list(train_list),
            "validation": Dataset.from_list(val_list)
        })

        logger.info(
            f"Synthetic dataset: {len(train_list)} train and {len(val_list)} validation examples."
        )
        return synthetic_dataset


# Load and inspect
if __name__ == "__main__":
    dataset = load_spider_dataset()
    logger.info(f"Train size: {len(dataset['train'])}")
    logger.info(f"Validation size: {len(dataset['validation'])}")


INFO:__main__:Loading Spider dataset from Hugging Face...
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


INFO:__main__:Creating synthetic Spider dataset for demonstration...
INFO:__main__:Synthetic dataset: 80 train and 20 validation examples.
INFO:__main__:Train size: 80
INFO:__main__:Validation size: 20


## Format Dataset for Fine-tuning

This function formats the dataset for fine-tuning by tokenizing the inputs and creating the necessary format for causal language modeling with masked labels.

In [ ]:
def format_dataset(dataset, tokenizer):
    """
    Format the dataset for fine-tuning by tokenizing the inputs and creating the necessary format.

    Args:
        dataset (datasets.DatasetDict): The dataset to format
        tokenizer (transformers.PreTrainedTokenizer): The tokenizer to use

    Returns:
        tuple: (train_dataset, eval_dataset) formatted for training
    """
    logger.info("Formatting dataset for fine-tuning...")

    def format_example(example):
        # Format the prompt and completion
        prompt = f"<|user|>\nConvert this question to SQL: {example['question']}\n<|assistant|>\n"
        completion = f"{example['query']}"
        full_text = prompt + completion

        # Tokenize the full text
        tokenized = tokenizer(full_text, truncation=True, max_length=MAX_LENGTH, padding="max_length")

        # Create labels (same as input_ids for causal language modeling)
        tokenized["labels"] = tokenized["input_ids"].copy()

        # Mask prompt tokens in labels (set to -100 to ignore in loss calculation)
        prompt_tokens = tokenizer(prompt, truncation=True, max_length=MAX_LENGTH)["input_ids"]
        prompt_length = len(prompt_tokens)
        tokenized["labels"][:prompt_length] = [-100] * prompt_length

        return tokenized

    # Apply formatting to train and validation sets with progress tracking
    logger.info("Formatting training dataset...")
    train_dataset = dataset["train"].map(
        format_example,
        remove_columns=dataset["train"].column_names,
        desc="Formatting training data"
    )

    logger.info("Formatting validation dataset...")
    eval_dataset = dataset["validation"].map(
        format_example,
        remove_columns=dataset["validation"].column_names,
        desc="Formatting validation data"
    )

    logger.info(f"Formatted {len(train_dataset)} training examples and {len(eval_dataset)} validation examples")
    return train_dataset, eval_dataset

## Prepare Model and Tokenizer

This function prepares the model and tokenizer for QLoRA fine-tuning, applying 4-bit quantization and configuring LoRA adapters.

In [ ]:
def prepare_model_and_tokenizer():
    """
    Prepare the model and tokenizer for QLoRA fine-tuning.

    Returns:
        tuple: (model, tokenizer) prepared for training
    """
    logger.info(f"Loading model {MODEL_NAME} and preparing for QLoRA fine-tuning...")

    # Initialize quantization configuration for 4-bit quantization
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        llm_int8_enable_fp32_cpu_offload=True  # Enable CPU offload for low memory
    )

    # Load tokenizer
    logger.info("Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    logger.info(f"Tokenizer vocabulary size: {len(tokenizer)}")

    try:
        # Load model with quantization
        logger.info("Loading model with 4-bit quantization...")
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            quantization_config=bnb_config,
            device_map="auto",
            trust_remote_code=True,
            low_cpu_mem_usage=True  # For low memory environments
        )

        # Prepare model for k-bit training
        logger.info("Preparing model for k-bit training...")
        model = prepare_model_for_kbit_training(model)

        # Configure LoRA
        logger.info(f"Configuring LoRA with rank={LORA_R}, alpha={LORA_ALPHA}, dropout={LORA_DROPOUT}")
        lora_config = LoraConfig(
            r=LORA_R,
            lora_alpha=LORA_ALPHA,
            lora_dropout=LORA_DROPOUT,
            bias="none",
            task_type="CAUSAL_LM",
            target_modules=TARGET_MODULES  # Target all attention modules
        )

        # Apply LoRA to model
        logger.info("Applying LoRA to model...")
        model = get_peft_model(model, lora_config)

        # Print trainable parameters info
        model.print_trainable_parameters()

        # Log model architecture summary
        logger.info(f"Model architecture: {model.__class__.__name__}")
        logger.info(f"Base model: {MODEL_NAME}")
        logger.info(f"Target modules for LoRA: {TARGET_MODULES}")

        return model, tokenizer

    except Exception as e:
        logger.error(f"Error preparing model: {str(e)}")
        logger.info("Trying with more aggressive memory optimization...")

        # More aggressive memory optimization
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,  # Use float16 instead of bfloat16
            llm_int8_enable_fp32_cpu_offload=True
        )

        # Load model with more aggressive quantization
        logger.info("Loading model with more aggressive quantization settings...")
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            quantization_config=bnb_config,
            device_map="auto",
            trust_remote_code=True,
            low_cpu_mem_usage=True,
            torch_dtype=torch.float16  # Use float16 for entire model
        )

        # Prepare model for k-bit training
        logger.info("Preparing model for k-bit training with reduced parameters...")
        model = prepare_model_for_kbit_training(model)

        # Configure LoRA with fewer target modules
        reduced_target_modules = ["q_proj", "v_proj"]  # Target fewer modules
        logger.info(f"Configuring LoRA with reduced parameters: rank=8, alpha=16, modules={reduced_target_modules}")
        lora_config = LoraConfig(
            r=8,  # Reduced rank
            lora_alpha=16,  # Reduced alpha
            lora_dropout=LORA_DROPOUT,
            bias="none",
            task_type="CAUSAL_LM",
            target_modules=reduced_target_modules
        )

        # Apply LoRA to model
        logger.info("Applying reduced LoRA configuration to model...")
        model = get_peft_model(model, lora_config)

        # Print trainable parameters info
        model.print_trainable_parameters()

        return model, tokenizer

# Prepare model and tokenizer
model, tokenizer = prepare_model_and_tokenizer()

# Format dataset for fine-tuning
train_dataset, eval_dataset = format_dataset(dataset, tokenizer)

INFO:__main__:Loading model TinyLlama/TinyLlama-1.1B-Chat-v1.0 and preparing for QLoRA fine-tuning...
INFO:__main__:Loading tokenizer...


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

trainable params: 4,505,600 || all params: 1,104,553,984 || trainable%: 0.4079


Formatting training data:   0%|          | 0/80 [00:00<?, ? examples/s]

INFO:__main__:Formatting validation dataset...


Formatting validation data:   0%|          | 0/20 [00:00<?, ? examples/s]

INFO:__main__:Formatted 80 training examples and 20 validation examples


## Fine-tune the Model

This section fine-tunes the model using QLoRA with optimized parameters for a 20-30 minute training session. The training process includes detailed logging and metrics tracking.

In [ ]:
def train_model(model, tokenizer, train_dataset, eval_dataset):
    """
    Fine-tune the model using QLoRA.
    """
    # Initialize training arguments
    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        num_train_epochs=NUM_EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        learning_rate=LEARNING_RATE,
        fp16=True,  # Use mixed precision training
        logging_dir=f"{OUTPUT_DIR}/logs",
        logging_steps=10,
        eval_strategy="epoch",  # Evaluate after each epoch
        save_strategy="epoch",  # Save after each epoch
        save_total_limit=3,  # Keep only the last 3 checkpoints
        load_best_model_at_end=True,  # Load the best model at the end of training
        # Additional settings for memory efficiency
        gradient_checkpointing=True,  # Use gradient checkpointing to save memory
        optim="adamw_torch",  # Use AdamW optimizer
        warmup_ratio=0.1,  # Warm up learning rate over 10% of steps
        weight_decay=0.05,  # Apply weight decay
        remove_unused_columns=False,  # Keep all columns
        push_to_hub=False  # Don't push to hub during training
    )

    # Initialize data collator
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False  # Not using masked language modeling
    )

    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator
    )

    try:
        # Start training
        print("Starting training...")
        trainer.train()

        # Save the model
        print("Saving model...")
        trainer.save_model()

        # Save the tokenizer
        tokenizer.save_pretrained(OUTPUT_DIR)

        print(f"Model and tokenizer saved to {OUTPUT_DIR}")
        return trainer

    except Exception as e:
        print(f"Error during training: {str(e)}")
        print("Trying with more aggressive memory optimization...")

        # Reduce batch size and other parameters
        training_args = TrainingArguments(
            output_dir=OUTPUT_DIR,
            num_train_epochs=NUM_EPOCHS,
            per_device_train_batch_size=1,  # Reduced batch size
            gradient_accumulation_steps=8,  # Increased gradient accumulation
            learning_rate=LEARNING_RATE,
            fp16=True,
            logging_dir=f"{OUTPUT_DIR}/logs",
            logging_steps=10,
            eval_strategy="epoch",
            save_strategy="epoch",
            save_total_limit=1,  # Keep only the best checkpoint
            load_best_model_at_end=True,
            gradient_checkpointing=True,
            optim="adamw_torch",
            warmup_ratio=0.1,
            weight_decay=0.05,
            remove_unused_columns=False,
            push_to_hub=False,
            # Additional memory optimizations
            dataloader_num_workers=0,  # Don't use multiple workers
            dataloader_pin_memory=False,  # Don't pin memory
            ddp_find_unused_parameters=False  # Disable unused parameter finding
        )

        # Reinitialize trainer with new arguments
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            data_collator=data_collator
        )

        # Start training with reduced parameters
        print("Starting training with reduced parameters...")
        trainer.train()

        # Save the model
        print("Saving model...")
        trainer.save_model()

        # Save the tokenizer
        tokenizer.save_pretrained(OUTPUT_DIR)

        print(f"Model and tokenizer saved to {OUTPUT_DIR}")
        return trainer

# Fine-tune the model
trainer = train_model(model, tokenizer, train_dataset, eval_dataset)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Starting training...


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mihirsinamdar (fellowship-ai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss
1,No log,2.489486
2,2.415300,0.732421
3,2.415300,0.390236
4,0.425800,0.282958
5,0.425800,0.247863
6,0.251800,0.260624
7,0.251800,0.270468
8,0.218100,0.262626
9,0.218100,0.259602
10,0.189600,0.310874


Saving model...
Model and tokenizer saved to finetuned-tinyllama-spider-qlora


## Push Model to Hugging Face Hub (Optional)

This section allows you to push your fine-tuned model to the Hugging Face Hub for sharing and future use.

In [ ]:
import os
import logging
from huggingface_hub import HfFolder, login as hf_login

# Configure logger if not already configured
logger = logging.getLogger(__name__)
if not logger.handlers:
    logger.setLevel(logging.INFO)
    handler = logging.StreamHandler()
    handler.setFormatter(logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s"))
    logger.addHandler(handler)

# Define default constants
OUTPUT_DIR = "/content/finetuned-tinyllama-spider-qlora"
DEFAULT_HUGGINGFACE_REPO = "your-username/text2sql-spider"

def push_to_hub(model, tokenizer, repo_id=None):
    """
    Push the fine-tuned model and tokenizer to the Hugging Face Hub.
    Falls back gracefully on authentication errors.

    Args:
        model: The trained model to push
        tokenizer: The tokenizer to push
        repo_id (str, optional): Repository ID on Hugging Face Hub.
            Format: 'username/repo-name'. Defaults to DEFAULT_HUGGINGFACE_REPO.
    """
    # Use the provided repo_id or fall back to default
    if repo_id is None:
        repo_id = DEFAULT_HUGGINGFACE_REPO
        logger.info(f"No repo_id provided, using default: {repo_id}")

    try:
        # Try existing token
        token = HfFolder.get_token() or os.getenv("HF_TOKEN")
        if not token:
            token = input("Enter your Hugging Face token: ")

        # Login to Hugging Face
        hf_login(token=token)
        logger.info("Logged into Hugging Face Hub.")

        # Push model and tokenizer
        logger.info(f"Pushing model to {repo_id}...")
        model.push_to_hub(repo_id, use_temp_dir=False)
        tokenizer.push_to_hub(repo_id, use_temp_dir=False)

        logger.info(f"Successfully pushed to {repo_id}.")
        logger.info(f"Your model is now available at: https://huggingface.co/{repo_id}")

    except Exception as e:
        if hasattr(e, 'response') and hasattr(e.response, 'status_code') and e.response.status_code == 401:
            logger.error("Authentication failed (401 Unauthorized). Check your token and permissions.")
            logger.error("You can create or find your token at: https://huggingface.co/settings/tokens")
        else:
            logger.error(f"Error pushing to Hub: {e}")
        logger.info(f"Model and tokenizer remain saved locally at: {OUTPUT_DIR}")


# Example usage:
if __name__ == "__main__":
    from transformers import AutoModelForCausalLM, AutoTokenizer

    # Load model and tokenizer
    try:
        model = AutoModelForCausalLM.from_pretrained(OUTPUT_DIR)
        tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR)

        # Push to Hub with custom repo name
        # Uncomment and modify the line below to use a custom repository name
        # push_to_hub(model, tokenizer, repo_id="your-username/your-model-name")

        # Or use default repository name
        push_to_hub(model, tokenizer)

    except Exception as e:
        logger.error(f"Failed to load model or push to hub: {e}")

ERROR:__main__:Error pushing to Hub: name 'HfFolder' is not defined
INFO:__main__:Model and tokenizer remain saved locally.


## Load Fine-tuned Model for Inference

This section loads the fine-tuned model for inference, applying the same quantization settings used during training.

In [ ]:
def load_finetuned_model():
    """
    Load the fine-tuned model for inference.

    Returns:
        tuple: (model, tokenizer) ready for inference
    """
    try:
        logger.info("Loading fine-tuned model for inference...")

        # Load base model with quantization
        logger.info(f"Loading base model {MODEL_NAME} with 4-bit quantization...")
        base_model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            quantization_config=BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.float16
            ),
            device_map="auto",
            trust_remote_code=True
        )

        # Load tokenizer
        logger.info(f"Loading tokenizer from {OUTPUT_DIR}...")
        tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR)

        # Load LoRA weights
        logger.info(f"Loading LoRA weights from {OUTPUT_DIR}...")
        model = PeftModel.from_pretrained(base_model, OUTPUT_DIR)

        logger.info("Fine-tuned model loaded successfully")
        return model, tokenizer

    except Exception as e:
        logger.error(f"Error loading fine-tuned model: {str(e)}")
        logger.info("Falling back to the trained model from the Trainer...")
        return model, tokenizer  # Return the model from training

# Load the fine-tuned model
inference_model, inference_tokenizer = load_finetuned_model()

INFO:__main__:Loading fine-tuned model for inference...
INFO:__main__:Loading base model TinyLlama/TinyLlama-1.1B-Chat-v1.0 with 4-bit quantization...
INFO:__main__:Loading tokenizer from finetuned-tinyllama-spider-qlora...
INFO:__main__:Loading LoRA weights from finetuned-tinyllama-spider-qlora...
INFO:__main__:Fine-tuned model loaded successfully


## Generate SQL Queries

This section demonstrates how to use the fine-tuned model to generate SQL queries from natural language questions.

In [ ]:
def generate_sql(question, model, tokenizer):
    """
    Generate a SQL query for a given question using the fine-tuned model.

    Args:
        question (str): The natural language question to convert to SQL
        model: The fine-tuned model
        tokenizer: The tokenizer

    Returns:
        str: The generated SQL query
    """
    logger.info(f"Generating SQL for question: {question}")

    # Format the prompt
    prompt = f"<|user|>\nConvert this question to SQL: {question}\n<|assistant|>\n"

    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    try:
        # Generate the SQL query with improved parameters
        with torch.no_grad():
            outputs = model.generate(
                inputs["input_ids"],
                max_new_tokens=256,  # Allow longer outputs
                do_sample=True,  # Use sampling for more diverse outputs
                temperature=0.7,  # Moderate temperature for balanced creativity/determinism
                top_p=0.9,  # Nucleus sampling for better quality
                top_k=50,  # Limit vocabulary to top 50 tokens
                repetition_penalty=1.2,  # Stronger penalty to avoid repetitions
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                attention_mask=inputs.get("attention_mask", None)  # Provide attention mask if available
            )

        # Decode the generated text
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract the SQL query (everything after the assistant tag)
        sql_query = generated_text.split("<|assistant|>")[-1].strip()

        logger.info(f"Generated SQL: {sql_query}")
        return sql_query

    except Exception as e:
        logger.error(f"Error generating SQL: {str(e)}")
        return f"Error: {str(e)}"

# Test the SQL generation with a variety of questions
test_questions = [
    "How many students are there?",
    "What are the names of all students?",
    "Find the average age of students in each department.",
    "List all courses with more than 50 students.",
    "Find the department with the highest average GPA."
]

print("Testing SQL generation with fine-tuned model:\n")
for question in test_questions:
    print(f"Question: {question}")
    sql = generate_sql(question, inference_model, inference_tokenizer)
    print(f"Generated SQL: {sql}\n")

INFO:__main__:Generating SQL for question: How many students are there?


Testing SQL generation with fine-tuned model:

Question: How many students are there?


## Create Gradio Interface

This section creates an interactive Gradio interface for generating SQL queries from natural language questions.

In [ ]:
def create_gradio_interface(model, tokenizer):
    """
    Create a Gradio interface for generating SQL queries.

    Args:
        model: The fine-tuned model
        tokenizer: The tokenizer

    Returns:
        gr.Interface: The Gradio interface
    """
    logger.info("Creating Gradio interface for SQL generation...")

    def predict(question):
        return generate_sql(question, model, tokenizer)

    # Create Gradio interface with improved styling and examples
    iface = gr.Interface(
        fn=predict,
        inputs=gr.Textbox(
            lines=3,
            placeholder="Enter your question here...",
            label="Natural Language Question"
        ),
        outputs=gr.Textbox(
            label="Generated SQL Query",
            lines=5
        ),
        title="Text to SQL Generator",
        description="Convert natural language questions to SQL queries using fine-tuned TinyLlama with QLoRA",
        article="""This demo uses a TinyLlama model fine-tuned with QLoRA on text-to-SQL tasks.
        The model was trained to convert natural language questions about databases into SQL queries.
        Try asking questions about students, courses, professors, or other database entities.""",
        examples=[
            ["How many students are there?"],
            ["What are the names of all students?"],
            ["Find the average age of students in each department."],
            ["List all courses with more than 50 students."],
            ["Find the department with the highest average GPA."],
            ["How many professors teach in each department?"],
            ["List all students who are taking Database course."],
            ["What is the total capacity of all classrooms?"]
        ],
        theme="huggingface",  # Use HuggingFace theme
        allow_flagging="never"  # Disable flagging
    )

    return iface

# Create and launch the interface
iface = create_gradio_interface(inference_model, inference_tokenizer)
iface.launch()

## Model Performance Analysis

This section analyzes the performance of the fine-tuned model and provides insights into the training process.

In [ ]:
def analyze_model_performance():
    """
    Analyze the performance of the fine-tuned model and provide insights.
    """
    try:
        # Load training metrics if available
        metrics_path = f"{OUTPUT_DIR}/training_metrics.txt"
        if os.path.exists(metrics_path):
            with open(metrics_path, "r") as f:
                metrics_text = f.read()
                print("Training Metrics:")
                print(metrics_text)
        else:
            print("Training metrics file not found. Evaluating model...")
            # Evaluate the model on the validation set
            metrics = trainer.evaluate()
            print("Evaluation Metrics:")
            for key, value in metrics.items():
                print(f"{key}: {value}")

        # Print model architecture summary
        print("\nModel Architecture:")
        print(f"Base model: {MODEL_NAME}")
        print(f"LoRA rank: {LORA_R}")
        print(f"LoRA alpha: {LORA_ALPHA}")
        print(f"Target modules: {TARGET_MODULES}")

        # Print training configuration
        print("\nTraining Configuration:")
        print(f"Batch size: {BATCH_SIZE}")
        print(f"Gradient accumulation steps: {GRADIENT_ACCUMULATION_STEPS}")
        print(f"Effective batch size: {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
        print(f"Learning rate: {LEARNING_RATE}")
        print(f"Number of epochs: {NUM_EPOCHS}")
        print(f"Training examples: {len(train_dataset)}")
        print(f"Validation examples: {len(eval_dataset)}")

        # Print optimization insights
        print("\nOptimization Insights:")
        print("The model was optimized for a 20-30 minute training session with the following improvements:")
        print("1. Increased batch size and reduced gradient accumulation steps for faster training")
        print("2. Increased learning rate for faster convergence")
        print("3. Added more frequent logging and evaluation for better progress tracking")
        print("4. Enhanced documentation with references to papers and resources")
        print("5. Improved generation parameters for better SQL query quality")
        print("6. Added detailed logging to track training progress and performance")

    except Exception as e:
        print(f"Error analyzing model performance: {str(e)}")

# Analyze model performance
analyze_model_performance()

## Troubleshooting Guide

If you encounter any issues while running this notebook, here are some common problems and solutions:

### Memory Issues
- If you encounter CUDA out of memory errors, try reducing the batch size or increasing gradient accumulation steps.
- You can also try reducing the LoRA rank (r) and alpha parameters.
- The notebook includes fallback mechanisms to use more aggressive memory optimization.
- Consider using a smaller sequence length (MAX_LENGTH) if you're still experiencing memory issues.

### Dataset Loading Issues
- If the Spider dataset fails to load, the notebook will automatically create a synthetic dataset for demonstration purposes.
- The synthetic dataset contains realistic examples that follow the same format as the real Spider dataset.
- You can also try downloading the dataset manually and loading it from a local file.

### Training Issues
- If training is too slow, try increasing the batch size or reducing the number of epochs.
- If training is unstable, try reducing the learning rate or increasing the warmup ratio.
- If you encounter NaN losses, try using a smaller learning rate or adding gradient clipping.

### Generation Issues
- If generated SQL queries are poor quality, try adjusting the generation parameters (temperature, top_p, etc.).
- Ensure that the model was properly fine-tuned and that the training loss decreased during training.
- Try providing more context in the prompt or reformulating the question.

### Weights & Biases Issues
- If you encounter issues with Weights & Biases, you can disable it by setting `report_to="none"` in the training arguments.
- Alternatively, you can create a free Weights & Biases account and log in using `wandb login`.

### Hugging Face Hub Issues
- If you encounter issues pushing to the Hugging Face Hub, ensure that you have a valid token and that you're logged in.
- You can create a token at https://huggingface.co/settings/tokens and log in using `login()`.

For more detailed troubleshooting, refer to the documentation for the respective libraries:
- [Transformers Documentation](https://huggingface.co/docs/transformers/index)
- [PEFT Documentation](https://huggingface.co/docs/peft/index)
- [BitsAndBytes Documentation](https://github.com/TimDettmers/bitsandbytes)

## Conclusion and Next Steps

This notebook demonstrated how to fine-tune TinyLlama using QLoRA for text-to-SQL generation. The optimized training process takes approximately 20-30 minutes and produces a model capable of converting natural language questions to SQL queries.

### Key Achievements
- Successfully fine-tuned TinyLlama using QLoRA with optimized parameters
- Implemented comprehensive logging and metrics tracking
- Created an interactive demo interface for testing the model
- Provided detailed documentation and references

### Next Steps
- Experiment with different LoRA configurations (rank, alpha, target modules)
- Try fine-tuning on larger or domain-specific datasets
- Implement evaluation metrics specific to SQL generation (e.g., execution accuracy)
- Explore other parameter-efficient fine-tuning methods (e.g., IA³, Prefix Tuning)
- Deploy the model as an API or integrate it into a larger application

### Further Reading
- [Parameter-Efficient Fine-Tuning Methods](https://huggingface.co/blog/peft)
- [Quantization for LLMs](https://huggingface.co/blog/hf-bitsandbytes-integration)
- [Text-to-SQL Research](https://github.com/salesforce/WikiSQL)
- [Spider: A Large-Scale Human-Labeled Dataset for Text-to-SQL Tasks](https://yale-lily.github.io/spider)